[View in Colaboratory](https://colab.research.google.com/github/roman807/TGS_Salt/blob/master/submission_final.ipynb)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model
from keras import models
from keras import optimizers
from keras import backend as K
from sklearn.metrics import confusion_matrix, log_loss
from google.colab import files
import tensorflow as tf
import cv2
import os
import json
import zipfile
import skimage.io as io
import skimage.transform as trans
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import h5py
import csv 

In [0]:
# ------------------- Load data (use kaggle API) ------------------- #
!pip install kaggle
api_token = {"username":"######","key":"############################"}
os.chdir('/')
!mkdir ~/.kaggle #kaggle API searches in root directory for .kaggle/kaggle.jso
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
# API link from Kaggle:
!kaggle competitions download -c tgs-salt-identification-challenge
zip_ref = zipfile.ZipFile('test.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
# ------------------- Prepare data ------------------- #
t_image_dir = os.chdir('/images')
test_im = os.listdir(t_image_dir)
x = np.array([np.array(cv2.imread(p, cv2.IMREAD_GRAYSCALE)) for p in test_im]) / 255
x = np.expand_dims(x, axis=3)

In [0]:
# ------------------- Authorize connection to google drive ------------------- #
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# ------------------- Load model from Google drive ------------------- #
last_model_file = drive.CreateFile({'id': '##############################'}) 
last_model_file.GetContentFile('model_3322rn.hdf5')
new_model = models.load_model('model_3322rn.hdf5')

In [0]:
# ------------------- Predict results ------------------- #
y_pred = new_model.predict(x, verbose=1)
y_pred_bin = np.round(y_pred + 0.0, 0) # <-- change threshold if necessary

In [0]:
# ------------------- Create string for submission ------------------- #
final_string = []
for element in y_pred_bin:
  locations = []
  counters = []
  loc_count = []
  array = np.transpose(element).flatten()
  if array[0]==1:
    locations.append(1)
    counters.append(1)
  for i in range(1,len(array)):
    if array[i] == 1:
      if array[i-1] == 0:
        locations.append(i+1)
        counters.append(1)
      else:
        counters[-1] += 1   
  for i in range(len(locations)):
    loc_count.append(locations[i])
    loc_count.append(counters[i])
  string = ' '.join(str(l) for l in loc_count)    
  final_string.append(string)

In [0]:
# ------------------- Download submission file ------------------- #
names = [word.split('.')[0] for word in test_im]
with open('submission_rn2.csv', 'w', newline ='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['id', 'rle_mask'])
  for i in range(len(names)):
    writer.writerow([names[i], final_string[i]])
files.download('submission_rn2.csv')

In [0]:
# ------------------- Save to google drive ------------------- #
uploaded = drive.CreateFile({'title': 'submission_rn2.csv'})
uploaded.SetContentFile('submission_rn2.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))